In [1]:
import tensorflow as tf
import numpy as np
import scipy.stats
import scipy.io
import scipy.sparse
from scipy.io import loadmat
import pandas as pd
import tensorflow_probability as tfp
tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn import preprocessing
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import BayesianRidge
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

2022-11-16 20:07:26.974139: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
xmiss = pd.read_csv('data_50/data_50_v1/bean/bean_clean.csv')

In [3]:
xfull = pd.read_csv('data_50/data_50_v1/bean/bean.csv')

In [4]:
mask = pd.read_csv('data_50/data_50_v1/bean/bean_sign.csv')

In [5]:
xhat_0 = np.copy(xmiss)
xhat_0[np.isnan(xmiss)] = 0

In [6]:
xhat_0 = np.array(xhat_0).astype(np.float32)
mask = np.array(mask).astype(np.float32)

In [7]:
d = 10 # 10D manifold
p_z = tfd.MultivariateNormalDiag(loc=tf.zeros(d, tf.float32))

2022-11-16 20:07:30.422947: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 20:07:31.173282: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10102 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:d6:00.0, compute capability: 8.6


In [8]:
n, p = xfull.shape

In [9]:
h = 128 # number of hidden units (same for all MLPs)

sigma = "relu"

decoder = tfk.Sequential([
  tfkl.InputLayer(input_shape=[d,]),
  tfkl.Dense(h, activation=sigma,kernel_initializer="he_normal"),
  tfkl.Dense(h, activation=sigma,kernel_initializer="he_normal"),
  tfkl.Dense(3*p,kernel_initializer="he_normal") # the decoder will output both the mean, the scale, and the number of degrees of freedoms (hence the 3*p)
])

In [10]:
iota = tf.Variable(np.zeros([1,p]),dtype=tf.float32) # this will be updated during training

In [11]:
encoder = tfk.Sequential([
  tfkl.InputLayer(input_shape=[p,]),
  tfkl.Dense(h, activation=sigma,kernel_initializer="he_normal"),
  tfkl.Dense(h, activation=sigma,kernel_initializer="he_normal"),
  tfkl.Dense(h, activation=sigma,kernel_initializer="he_normal"),
  tfkl.Dense(3*d,kernel_initializer="he_normal")
])

In [12]:
@tf.function
def miwae_bound(xhat_0_batch, mask_batch,K):

    tiledmask = tf.tile(mask_batch,[K,1])
    mask_complement_float = tf.abs(mask_batch-1)

    tilediota = tf.tile(iota,[xhat_0_batch.shape[0],1])
    iotax = xhat_0_batch + tf.multiply(tilediota,mask_complement_float)

    out_encoder = encoder(iotax)
    q_zgivenxobs = tfd.Independent(distribution=tfd.StudentT(loc=out_encoder[..., :d], scale=tf.nn.softplus(out_encoder[..., d:(2*d)]), df=3 + tf.nn.softplus(out_encoder[..., (2*d):(3*d)])))
    zgivenx = q_zgivenxobs.sample(K)
    zgivenx_flat = tf.reshape(zgivenx,[K*xhat_0_batch.shape[0],d])

    out_decoder = decoder(zgivenx_flat)
    data_flat = tf.reshape(tf.tile(xhat_0_batch,[K,1]),[-1,1])
    all_means_obs_model = out_decoder[..., :p]
    all_scales_obs_model = tf.nn.softplus(out_decoder[..., p:(2*p)]) + 0.001
    all_degfreedom_obs_model = tf.nn.softplus(out_decoder[..., (2*p):(3*p)]) + 3
    all_log_pxgivenz_flat = tfd.StudentT(loc=tf.reshape(all_means_obs_model,[-1,1]),scale=tf.reshape(all_scales_obs_model,[-1,1]),df=tf.reshape(all_degfreedom_obs_model,[-1,1])).log_prob(data_flat)
    all_log_pxgivenz = tf.reshape(all_log_pxgivenz_flat,[K*xhat_0_batch.shape[0],p])

    logpxobsgivenz = tf.reshape(tf.reduce_sum(tf.multiply(all_log_pxgivenz,tiledmask),1),[K,xhat_0_batch.shape[0]])
    logpz = p_z.log_prob(zgivenx)
    logq = q_zgivenxobs.log_prob(zgivenx)

    average_miwae_bound = tf.reduce_mean(tf.reduce_logsumexp(logpxobsgivenz + logpz - logq,0) -tf.math.log(tf.cast(K,tf.float32)) ) 
    return average_miwae_bound

In [13]:
@tf.function
def miwae_impute(xhat_0_batch, mask_batch,L=1000,kind="single",num_samples=20):

    # kind = "single" will return only the single imputation
    # kind = "multiple" will return num_samples multiple imputations
    # kind = "both" will both the single imputation and num_samples multiple imputations

    tiledmask = tf.tile(mask_batch,[L,1])
    mask_complement_float = tf.abs(mask_batch-1)

    tilediota = tf.tile(iota,[xhat_0_batch.shape[0],1])
    iotax = xhat_0_batch + tf.multiply(tilediota,mask_complement_float)

    out_encoder = encoder(iotax)
    q_zgivenxobs = tfd.Independent(distribution=tfd.StudentT(loc=out_encoder[..., :d], scale=tf.nn.softplus(out_encoder[..., d:(2*d)]), df=3 + tf.nn.softplus(out_encoder[..., (2*d):(3*d)])))
    zgivenx = q_zgivenxobs.sample(L)
    zgivenx_flat = tf.reshape(zgivenx,[L*xhat_0_batch.shape[0],d])

    out_decoder = decoder(zgivenx_flat)
    data_flat = tf.reshape(tf.tile(xhat_0_batch,[L,1]),[-1,1])
    all_means_obs_model = out_decoder[..., :p]
    all_scales_obs_model = tf.nn.softplus(out_decoder[..., p:(2*p)]) + 0.001
    all_degfreedom_obs_model = tf.nn.softplus(out_decoder[..., (2*p):(3*p)]) + 3
    xgivenz = tfd.Independent(distribution=tfd.StudentT(loc=all_means_obs_model, scale=all_scales_obs_model, df=all_degfreedom_obs_model))
    all_log_pxgivenz_flat = tfd.StudentT(loc=tf.reshape(all_means_obs_model,[-1,1]),scale=tf.reshape(all_scales_obs_model,[-1,1]),df=tf.reshape(all_degfreedom_obs_model,[-1,1])).log_prob(data_flat)
    all_log_pxgivenz = tf.reshape(all_log_pxgivenz_flat,[L*xhat_0_batch.shape[0],p])

    logpxobsgivenz = tf.reshape(tf.reduce_sum(tf.multiply(all_log_pxgivenz,tiledmask),1),[L,xhat_0_batch.shape[0]])
    logpz = p_z.log_prob(zgivenx)
    logq = q_zgivenxobs.log_prob(zgivenx)

    log_imp_weights = logpxobsgivenz + logpz - logq # same importance wieghts used for single and multiple imputation

    if ((kind=="single") or (kind=="both")):

        imp_weights = tf.nn.softmax(log_imp_weights,0) # these are w_1,....,w_L for all observations in the batch
        xms = tf.reshape(xgivenz.mean(),[L,xhat_0_batch.shape[0],p])
        xm = xhat_0_batch + tf.multiply(tf.einsum('ki,kij->ij', imp_weights, xms),mask_complement_float)

    if ((kind=="multiple") or (kind=="both")):

        sir_logits = tf.transpose(log_imp_weights)
        sir = tfd.Categorical(logits = sir_logits).sample(num_samples)
        xmul = tf.reshape(xhat_0_batch + tf.multiply(tf.reshape(xgivenz.sample(),[L,xhat_0_batch.shape[0],p]),mask_complement_float),[L,xhat_0_batch.shape[0],p])
    if (kind=="single"):
        return xm

    if (kind=="multiple"):
        return tf.gather(tf.transpose(xmul,perm=[1,0,2]), tf.transpose(sir), axis = 1, batch_dims=1)

    if (kind=="both"):
        return xm, tf.gather(tf.transpose(xmul,perm=[1,0,2]), tf.transpose(sir), axis = 1, batch_dims=1)

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

In [15]:
all_parameters = encoder.trainable_variables + decoder.trainable_variables + [iota]

In [16]:
@tf.function
def gradient_step_miwae(xhat_0_batch, mask_batch,K):
      with tf.GradientTape() as tape: # the gradient tape saves all the step that needs to be saved fopr automatic differentiation
        loss = -  miwae_bound(xhat_0_batch, mask_batch,K)  # the loss is the negative ELBO
      gradients = tape.gradient(loss, all_parameters)  # here, the gradient is automatically computed
      optimizer.apply_gradients(zip(gradients, all_parameters))  # Adam iteration

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices((xhat_0,mask)).shuffle(n).batch(32) # TF creates the batches for us

In [18]:
EPOCHS = 100
xhat_miwae = np.copy(xhat_0)

for epoch in range(1,EPOCHS+1):
    for xhat_0_batch, mask_batch in train_dataset:
        gradient_step_miwae(xhat_0_batch, mask_batch,K = 20) # Adam iteration
  
    print('Epoch  %g' %epoch)
    
    if epoch == 100:
        for i in range(n):
            xhat_miwae[i,:]  = miwae_impute(xhat_0[i,:].reshape([1,p]), mask[i,:].reshape([1,p]), kind = "single",L = 10000).numpy()

Instructions for updating:
Please pass an integer value for `reinterpreted_batch_ndims`. The current behavior corresponds to `reinterpreted_batch_ndims=tf.size(distribution.batch_shape_tensor()) - 1`.


2022-11-16 20:07:34.919124: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch  1
Epoch  2
Epoch  3
Epoch  4
Epoch  5
Epoch  6
Epoch  7
Epoch  8
Epoch  9
Epoch  10
Epoch  11
Epoch  12
Epoch  13
Epoch  14
Epoch  15
Epoch  16
Epoch  17
Epoch  18
Epoch  19
Epoch  20
Epoch  21
Epoch  22
Epoch  23
Epoch  24
Epoch  25
Epoch  26
Epoch  27
Epoch  28
Epoch  29
Epoch  30
Epoch  31
Epoch  32
Epoch  33
Epoch  34
Epoch  35
Epoch  36
Epoch  37
Epoch  38
Epoch  39
Epoch  40
Epoch  41
Epoch  42
Epoch  43
Epoch  44
Epoch  45
Epoch  46
Epoch  47
Epoch  48
Epoch  49
Epoch  50
Epoch  51
Epoch  52
Epoch  53
Epoch  54
Epoch  55
Epoch  56
Epoch  57
Epoch  58
Epoch  59
Epoch  60
Epoch  61
Epoch  62
Epoch  63
Epoch  64
Epoch  65
Epoch  66
Epoch  67
Epoch  68
Epoch  69
Epoch  70
Epoch  71
Epoch  72
Epoch  73
Epoch  74
Epoch  75
Epoch  76
Epoch  77
Epoch  78
Epoch  79
Epoch  80
Epoch  81
Epoch  82
Epoch  83
Epoch  84
Epoch  85
Epoch  86
Epoch  87
Epoch  88
Epoch  89
Epoch  90
Epoch  91
Epoch  92
Epoch  93
Epoch  94
Epoch  95
Epoch  96
Epoch  97
Epoch  98
Epoch  99
Epoch  100


In [19]:
output = pd.DataFrame(xhat_miwae, columns = xmiss.columns)

In [20]:
output.to_csv('MIWAE_50.csv', index = False)